In [1]:
from io import StringIO
from pathlib import Path
import time
import os

import requests
import pandas as pd

from cs_kit.exceptions import APIException


class ComputeStudio:
    '''
    Python class that wraps Compute Studio REST API.
    See the cell below for an example API call.
    '''
    host = "https://compute.studio"

    def __init__(self, owner, title, api_token=None):
        self.owner = owner
        self.title = title
        api_token = self.get_token(api_token)
        self.auth_header = {"Authorization": f"Token {api_token}"}
        self.sim_url = f"{self.host}/{owner}/{title}/api/v1/"
        self.inputs_url = f"{self.host}/{owner}/{title}/api/v1/inputs/"

    def create(self, adjustment: dict = None, meta_parameters: dict = None):
        adjustment = adjustment or {}
        meta_parameters = meta_parameters or {}
        resp = requests.post(
            self.sim_url,
            json={"adjustment": adjustment, "meta_parameters": meta_parameters},
            headers=self.auth_header,
        )
        if resp.status_code == 201:
            data = resp.json()
            pollresp = requests.get(
                f"{self.sim_url}{data['sim']['model_pk']}/edit/",
                headers=self.auth_header,
            )
            polldata = pollresp.json()
            while pollresp.status_code == 200 and polldata["status"] == "PENDING":
                time.sleep(3)
                pollresp = requests.get(
                    f"{self.sim_url}{data['sim']['model_pk']}/edit/",
                    headers=self.auth_header,
                )
                polldata = pollresp.json()
            if pollresp.status_code == 200 and polldata["status"] == "SUCCESS":
                simresp = requests.get(
                    f"{self.sim_url}{data['sim']['model_pk']}/remote/",
                    headers=self.auth_header,
                )
                return simresp.json()
            else:
                raise APIException(pollresp.text)
        raise APIException(resp.text)

    def detail(self, model_pk):
        while True:
            resp = requests.get(f"{self.sim_url}{model_pk}/", headers=self.auth_header)
            if resp.status_code == 202:
                pass
            elif resp.status_code == 200:
                return resp.json()
            else:
                raise APIException(resp.text)
            time.sleep(5)

    def results(self, model_pk):
        result = self.detail(model_pk)
        res = {}
        for output in result["outputs"]["downloadable"]:
            if output["media_type"] == "CSV":
                res[output["title"]] = pd.read_csv(StringIO(output["data"]))
            else:
                print(f'{output["media_type"]} not implemented yet')
        return res

    def get_token(self, api_token):
        token_file_path = Path.home() / ".cs-api-token"
        if api_token:
            return api_token
        elif os.environ.get("CS_API_TOKEN", None) is not None:
            return os.environ["CS_API_TOKEN"]
        elif token_file_path.exists():
            with open(token_file_path, "r") as f:
                return f.read().strip()
        else:
            raise APIException(
                f"API token not found. It can be passed as an argument to "
                f"this class, as an environment variable at CS_API_TOKEN, "
                f"or read from {token_file_path}"
            )

    def update_sim(self, model_pk, **sim_kwargs):
        """
        Update meta data about a simulation. For now, you can set:
        - title: title of the simulation.
        - is_public: visibility of the simulation.
        - notify_on_completion: set to True if you'd like an email when
          the simulation completes.
        Ex:
        cs.update_sim(
            model_pk=123,
            title="hello world",
            is_public=True,
            notify_on_completion=True
        )
        """
        extra_kwargs = sim_kwargs.keys() - {
            "title",
            "is_public",
            "notify_on_completion",
        }
        if extra_kwargs:
            raise APIException(
                f"Extra arguments were given to update_sim: {extra_kwargs}"
            )
        resp = requests.put(
            f"{self.sim_url}{model_pk}/", json=sim_kwargs, headers=self.auth_header,
        )
        if resp.status_code == 200:
            return resp.json()
        else:
            raise APIException(resp.text)

In [2]:
# To retrieve your authentication token: https://docs.compute.studio/api/auth/
api = ComputeStudio("Peter-Metz", "geoweight", api_token="YOUR_TOKEN")

In [6]:
# Create your simulation, adjusting any of the inputs below
res = api.create(
    adjustment = {
        "Prepare Solver": {
            "A01750_targ": [{"value": True}],
            "A04800_targ": [{"value": True}],
            "A02500_targ": [{"value": True}],
            "A01000_targ": [{"value": True}],
            "A00600_targ": [{"value": True}],
            "A00300_targ": [{"value": True}],
            "A00200_targ": [{"value": True}],
            "A00100_targ": [{"value": True}],
            "MARS2_targ": [{"value": True}],
            "MARS1_targ": [{"value": True}],
            "N1_targ": [{"value": True}],
            "AGI_STUB": [{"value": 6}],
            "max_nfev": [{"value": 50}],
            "ftol": [{"value": 0.0001}]
        }
    }
)

In [10]:
# View the state weights as a dataframe
# The "renderable" CS outputs cannot be accessed with the API.
# See this output page for an example of "renderable" outputs: https://compute.studio/Peter-Metz/geoweight/15/
result = api.results(res["model_pk"])
result["state_weights"]

,Unnamed: 0,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,TX,UT,VT,VA,WA,WV,WI,WY,OA,PR
0,0,18.513756,3.291967,22.609103,12.226915,90.079062,22.096323,11.146854,3.684259,0.680014,...,87.580944,19.048329,3.512185,29.351811,28.769721,8.788595,30.198700,3.585664,5.353417,0.434610
1,1,11.085242,1.971088,13.537360,7.320951,53.935474,13.230329,6.674258,2.205976,0.407163,...,52.439708,11.405321,2.102946,17.574604,17.226074,5.262233,18.081685,2.146941,3.205396,0.260226
2,2,18.894109,5.585192,34.265744,10.076201,227.303852,39.610387,26.964763,6.363313,7.060505,...,134.779750,12.842584,3.716573,55.909166,55.498289,8.036921,31.552094,3.919525,6.531027,0.709026
3,3,17.655551,6.738838,33.968415,9.144289,239.285972,40.080755,32.419873,7.814053,16.213608,...,123.007188,10.476562,3.971675,57.954967,56.959299,7.888085,27.320374,4.245332,7.855540,0.947254
4,4,9.593749,1.908725,16.662852,4.426363,147.994195,17.607537,11.738130,2.375460,1.665807,...,86.898026,5.860404,1.263195,27.776435,26.282445,3.371586,15.384971,1.358244,3.344063,0.186312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18297,18297,0.482701,0.253669,1.025631,0.325419,5.485853,1.549439,0.424609,0.124137,0.030951,...,3.226549,0.345427,0.115598,1.151399,1.515777,0.199344,1.192702,0.109293,0.011794,0.007857
18298,18298,5.365655,0.796758,8.131383,4.433171,25.358357,10.791895,1.145768,0.342876,0.002456,...,34.755174,8.025806,0.669141,6.833883,9.069614,1.777683,19.544754,0.607003,0.028680,0.014631
18299,18299,4.254382,0.631742,6.447305,3.515024,20.106427,8.556803,0.908470,0.271864,0.001947,...,27.557085,6.363594,0.530557,5.418529,7.191220,1.409510,15.496871,0.481288,0.022740,0.011601
18300,18300,3.911673,0.580853,5.927946,3.231873,18.486766,7.867515,0.835289,0.249964,0.001790,...,25.337240,5.850978,0.487818,4.982042,6.611936,1.295968,14.248529,0.442518,0.020909,0.010666
